# Artificial Intelligence Nanodegree
## Machine Translation Project
In this notebook, sections that end with **'(IMPLEMENTATION)'** in the header indicate that the following blocks of code will require additional functionality which you must provide. Please be sure to read the instructions carefully!

## Introduction
In this notebook, you will build a deep neural network that functions as part of an end-to-end machine translation pipeline. Your completed pipeline will accept English text as input and return the French translation.

- **Preprocess** - You'll convert text to sequence of integers.
- **Models** Create models which accepts a sequence of integers as input and returns a probability distribution over possible translations. After learning about the basic types of neural networks that are often used for machine translation, you will engage in your own investigations, to design your own model!
- **Prediction** Run the model on English text.

In [1]:
%load_ext autoreload
%aimport helper, project_tests
%autoreload 1

Using TensorFlow backend.


In [2]:
import collections

import helper
import numpy as np
import project_tests as tests

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
# from keras.utils import pad_sequences
from keras.models import Model
from keras.layers import GRU, Input, Dense, TimeDistributed, Activation, RepeatVector, Bidirectional
from keras.layers.embeddings import Embedding
# from keras.layers import Embedding
from keras.optimizers import Adam
from keras.losses import sparse_categorical_crossentropy

### Verify access to the GPU
The following test applies only if you expect to be using a GPU, e.g., while running in a Udacity Workspace or using an AWS instance with GPU support. Run the next cell, and verify that the device_type is "GPU".
- If the device is not GPU & you are running from a Udacity Workspace, then save your workspace with the icon at the top, then click "enable" at the bottom of the workspace.
- If the device is not GPU & you are running from an AWS instance, then refer to the cloud computing instructions in the classroom to verify your setup steps.

In [3]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 18442248471910592117
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 16376205661365267822
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 6010408163576475266
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 1466598674102779369
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7160266752
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6456869295845120607
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2060 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5"
, name: "/device:GPU:1"
device_type: "GPU"
memory_limit: 2351366144
locality {
  bus_id: 1
  links {
  }
}
incarna

## Dataset
We begin by investigating the dataset that will be used to train and evaluate your pipeline.  The most common datasets used for machine translation are from [WMT](http://www.statmt.org/).  However, that will take a long time to train a neural network on.  We'll be using a dataset we created for this project that contains a small vocabulary.  You'll be able to train your model in a reasonable time with this dataset.
### Load Data
The data is located in `data/small_vocab_en` and `data/small_vocab_fr`. The `small_vocab_en` file contains English sentences with their French translations in the `small_vocab_fr` file. Load the English and French data from these files from running the cell below.

In [4]:
# Load English data
english_sentences = helper.load_data('data/small_vocab_en')
# Load French data
french_sentences = helper.load_data('data/small_vocab_fr')

print('Dataset Loaded')

Dataset Loaded


### Files
Each line in `small_vocab_en` contains an English sentence with the respective translation in each line of `small_vocab_fr`.  View the first two lines from each file.

In [5]:
for sample_i in range(2):
    print('small_vocab_en Line {}:  {}'.format(sample_i + 1, english_sentences[sample_i]))
    print('small_vocab_fr Line {}:  {}'.format(sample_i + 1, french_sentences[sample_i]))

small_vocab_en Line 1:  new jersey is sometimes quiet during autumn , and it is snowy in april .
small_vocab_fr Line 1:  new jersey est parfois calme pendant l' automne , et il est neigeux en avril .
small_vocab_en Line 2:  the united states is usually chilly during july , and it is usually freezing in november .
small_vocab_fr Line 2:  les états-unis est généralement froid en juillet , et il gèle habituellement en novembre .


From looking at the sentences, you can see they have been preprocessed already.  The puncuations have been delimited using spaces. All the text have been converted to lowercase.  This should save you some time, but the text requires more preprocessing.
### Vocabulary
The complexity of the problem is determined by the complexity of the vocabulary.  A more complex vocabulary is a more complex problem.  Let's look at the complexity of the dataset we'll be working with.

In [6]:
english_words_counter = collections.Counter([word for sentence in english_sentences for word in sentence.split()])
french_words_counter = collections.Counter([word for sentence in french_sentences for word in sentence.split()])

print('{} English words.'.format(len([word for sentence in english_sentences for word in sentence.split()])))
print('{} unique English words.'.format(len(english_words_counter)))
print('10 Most common words in the English dataset:')
print('"' + '" "'.join(list(zip(*english_words_counter.most_common(10)))[0]) + '"')
print()
print('{} French words.'.format(len([word for sentence in french_sentences for word in sentence.split()])))
print('{} unique French words.'.format(len(french_words_counter)))
print('10 Most common words in the French dataset:')
print('"' + '" "'.join(list(zip(*french_words_counter.most_common(10)))[0]) + '"')

1823250 English words.
227 unique English words.
10 Most common words in the English dataset:
"is" "," "." "in" "it" "during" "the" "but" "and" "sometimes"

1961295 French words.
355 unique French words.
10 Most common words in the French dataset:
"est" "." "," "en" "il" "les" "mais" "et" "la" "parfois"


For comparison, _Alice's Adventures in Wonderland_ contains 2,766 unique words of a total of 15,500 words.
## Preprocess
For this project, you won't use text data as input to your model. Instead, you'll convert the text into sequences of integers using the following preprocess methods:
1. Tokenize the words into ids
2. Add padding to make all the sequences the same length.

Time to start preprocessing the data...
### Tokenize (IMPLEMENTATION)
For a neural network to predict on text data, it first has to be turned into data it can understand. Text data like "dog" is a sequence of ASCII character encodings.  Since a neural network is a series of multiplication and addition operations, the input data needs to be number(s).

We can turn each character into a number or each word into a number.  These are called character and word ids, respectively.  Character ids are used for character level models that generate text predictions for each character.  A word level model uses word ids that generate text predictions for each word.  Word level models tend to learn better, since they are lower in complexity, so we'll use those.

Turn each sentence into a sequence of words ids using Keras's [`Tokenizer`](https://keras.io/preprocessing/text/#tokenizer) function. Use this function to tokenize `english_sentences` and `french_sentences` in the cell below.

Running the cell will run `tokenize` on sample data and show output for debugging.

In [7]:
def tokenize(x):
    """
    Tokenize x
    :param x: List of sentences/strings to be tokenized
    :return: Tuple of (tokenized x data, tokenizer used to tokenize x)
    """
    # TODO: Implement
    x_tk = Tokenizer()
    x_tk.fit_on_texts(x)
    return x_tk.texts_to_sequences(x), x_tk

tests.test_tokenize(tokenize)

# Tokenize Example output
text_sentences = [
    'The quick brown fox jumps over the lazy dog .',
    'By Jove , my quick study of lexicography won a prize .',
    'This is a short sentence .']
text_tokenized, text_tokenizer = tokenize(text_sentences)
print(text_tokenizer.word_index)
print()
for sample_i, (sent, token_sent) in enumerate(zip(text_sentences, text_tokenized)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(sent))
    print('  Output: {}'.format(token_sent))

{'the': 1, 'quick': 2, 'a': 3, 'brown': 4, 'fox': 5, 'jumps': 6, 'over': 7, 'lazy': 8, 'dog': 9, 'by': 10, 'jove': 11, 'my': 12, 'study': 13, 'of': 14, 'lexicography': 15, 'won': 16, 'prize': 17, 'this': 18, 'is': 19, 'short': 20, 'sentence': 21}

Sequence 1 in x
  Input:  The quick brown fox jumps over the lazy dog .
  Output: [1, 2, 4, 5, 6, 7, 1, 8, 9]
Sequence 2 in x
  Input:  By Jove , my quick study of lexicography won a prize .
  Output: [10, 11, 12, 2, 13, 14, 15, 16, 3, 17]
Sequence 3 in x
  Input:  This is a short sentence .
  Output: [18, 19, 3, 20, 21]


### Padding (IMPLEMENTATION)
When batching the sequence of word ids together, each sequence needs to be the same length.  Since sentences are dynamic in length, we can add padding to the end of the sequences to make them the same length.

Make sure all the English sequences have the same length and all the French sequences have the same length by adding padding to the **end** of each sequence using Keras's [`pad_sequences`](https://keras.io/preprocessing/sequence/#pad_sequences) function.

In [8]:
def pad(x, length=None):
    """
    Pad x
    :param x: List of sequences.
    :param length: Length to pad the sequence to.  If None, use length of longest sequence in x.
    :return: Padded numpy array of sequences
    """
    # TODO: Implement
    if length is None:
        length = max([len(sentence) for sentence in x])
    return pad_sequences(x, maxlen=length, padding='post', truncating='post')
    
tests.test_pad(pad)

# Pad Tokenized output
test_pad = pad(text_tokenized)
for sample_i, (token_sent, pad_sent) in enumerate(zip(text_tokenized, test_pad)):
    print('Sequence {} in x'.format(sample_i + 1))
    print('  Input:  {}'.format(np.array(token_sent)))
    print('  Output: {}'.format(pad_sent))

Sequence 1 in x
  Input:  [1 2 4 5 6 7 1 8 9]
  Output: [1 2 4 5 6 7 1 8 9 0]
Sequence 2 in x
  Input:  [10 11 12  2 13 14 15 16  3 17]
  Output: [10 11 12  2 13 14 15 16  3 17]
Sequence 3 in x
  Input:  [18 19  3 20 21]
  Output: [18 19  3 20 21  0  0  0  0  0]


### Preprocess Pipeline
Your focus for this project is to build neural network architecture, so we won't ask you to create a preprocess pipeline.  Instead, we've provided you with the implementation of the `preprocess` function.

In [9]:
def preprocess(x, y):
    """
    Preprocess x and y
    :param x: Feature List of sentences
    :param y: Label List of sentences
    :return: Tuple of (Preprocessed x, Preprocessed y, x tokenizer, y tokenizer)
    """
    preprocess_x, x_tk = tokenize(x)
    preprocess_y, y_tk = tokenize(y)

    preprocess_x = pad(preprocess_x)
    preprocess_y = pad(preprocess_y)

    # Keras's sparse_categorical_crossentropy function requires the labels to be in 3 dimensions
    preprocess_y = preprocess_y.reshape(*preprocess_y.shape, 1)

    return preprocess_x, preprocess_y, x_tk, y_tk

preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer =\
    preprocess(english_sentences, french_sentences)
    
max_english_sequence_length = preproc_english_sentences.shape[1]
max_french_sequence_length = preproc_french_sentences.shape[1]
english_vocab_size = len(english_tokenizer.word_index)
french_vocab_size = len(french_tokenizer.word_index)

print('Data Preprocessed')
print("Max English sentence length:", max_english_sequence_length)
print("Max French sentence length:", max_french_sequence_length)
print("English vocabulary size:", english_vocab_size)
print("French vocabulary size:", french_vocab_size)

Data Preprocessed
Max English sentence length: 15
Max French sentence length: 21
English vocabulary size: 199
French vocabulary size: 344


### Split train and test data

In [140]:
from sklearn.model_selection import train_test_split

(train_english_sentences, test_english_sentences, 
train_french_sentences, test_french_sentences) = train_test_split(
    preproc_english_sentences, preproc_french_sentences, test_size=0.33, random_state=0)

## Models
In this section, you will experiment with various neural network architectures.
You will begin by training four relatively simple architectures.
- Model 1 is a simple RNN
- Model 2 is a RNN with Embedding
- Model 3 is a Bidirectional RNN
- Model 4 is an Encoder-Decoder RNN

After experimenting with the four simple architectures, you will construct a deeper architecture that is designed to outperform all four models.
### Ids Back to Text
The neural network will be translating the input to words ids, which isn't the final form we want.  We want the French translation.  The function `logits_to_text` will bridge the gab between the logits from the neural network to the French translation.  You'll be using this function to better understand the output of the neural network.

In [105]:
def logits_to_text(logits, tokenizer):
    """
    Turn logits from a neural network into text using the tokenizer
    :param logits: Logits from a neural network
    :param tokenizer: Keras Tokenizer fit on the labels
    :return: String that represents the text of the logits
    """
    index_to_words = {id: word for word, id in tokenizer.word_index.items()}
    index_to_words[0] = '<PAD>'

    return ' '.join([index_to_words[prediction] for prediction in np.argmax(logits, 1)])

print('`logits_to_text` function loaded.')

`logits_to_text` function loaded.


### Model 1: RNN (IMPLEMENTATION)
![RNN](images/rnn.png)
A basic RNN model is a good baseline for sequence data.  In this model, you'll build a RNN that translates English to French.

In [147]:
def simple_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a basic RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Build the layers
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    rnn = GRU(64, return_sequences=True)(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    
tests.test_simple_model(simple_model)

# Reshaping the input to work with a basic RNN
tmp_x = pad(train_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# Train the neural network
simple_rnn_model = simple_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_rnn_model.fit(tmp_x, train_french_sentences, batch_size=1024, epochs=50, validation_split=0.2)

# Print prediction(s)
print(logits_to_text(simple_rnn_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 73892 samples, validate on 18474 samples
Epoch 1/50
73892/73892 [==============================] - 13s 177us/step - loss: 3.8443 - acc: 0.3915 - val_loss: 2.7320 - val_acc: 0.4130
Epoch 2/50
73892/73892 [==============================] - 2s 22us/step - loss: 2.5997 - acc: 0.4523 - val_loss: 2.5188 - val_acc: 0.4682
Epoch 3/50
73892/73892 [==============================] - 2s 22us/step - loss: 2.4415 - acc: 0.4742 - val_loss: 2.3719 - val_acc: 0.4738
Epoch 4/50
73892/73892 [==============================] - 2s 23us/step - loss: 2.2908 - acc: 0.4928 - val_loss: 2.2167 - val_acc: 0.5075
Epoch 5/50
73892/73892 [==============================] - 2s 22us/step - loss: 2.1343 - acc: 0.5154 - val_loss: 2.0640 - val_acc: 0.5275
Epoch 6/50
73892/73892 [==============================] - 2s 23us/step - loss: 1.9904 - acc: 0.5482 - val_loss: 1.9284 - val_acc: 0.5609
Epoch 7/50
73892/73892 [==============================] - 2s 23us/step - loss: 1.8660 - acc: 0.5706 - val_loss: 1.8186 - val_a

In [148]:
tmp_x = pad(test_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

_loss, _acc = simple_rnn_model.evaluate(tmp_x, test_french_sentences, batch_size=1024, verbose=True)
print(f'loss: {_loss:.3f}, accuracy: {_acc:.3f}')

45495/45495 [==============================] - 0s 8us/step
loss: nan, accuracy: 0.677


### Model 2: Embedding (IMPLEMENTATION)
![RNN](images/embedding.png)
You've turned the words into ids, but there's a better representation of a word.  This is called word embeddings.  An embedding is a vector representation of the word that is close to similar words in n-dimensional space, where the n represents the size of the embedding vectors.

In this model, you'll create a RNN model using embedding.

In [149]:
def embed_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a RNN model using word embedding on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    embedding_size = 32

    input_seq = Input(shape=input_shape[1:])
    input_emb = Embedding(english_vocab_size, embedding_size, input_length=input_shape[1])(input_seq)
    rnn = GRU(64, return_sequences=True)(input_emb)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_embed_model(embed_model)


# TODO: Reshape the input
tmp_x = pad(train_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, preproc_french_sentences.shape[-2]))

# TODO: Train the neural network
simple_embed_model = embed_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_embed_model.fit(tmp_x, train_french_sentences, batch_size=1024, epochs=50, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(simple_embed_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 73892 samples, validate on 18474 samples
Epoch 1/50
73892/73892 [==============================] - 13s 182us/step - loss: 4.2420 - acc: 0.3954 - val_loss: 3.1214 - val_acc: 0.4061
Epoch 2/50
73892/73892 [==============================] - 2s 24us/step - loss: 3.0314 - acc: 0.4085 - val_loss: 2.9486 - val_acc: 0.4062
Epoch 3/50
73892/73892 [==============================] - 2s 24us/step - loss: 2.7602 - acc: 0.4182 - val_loss: 2.5495 - val_acc: 0.4603
Epoch 4/50
73892/73892 [==============================] - 2s 24us/step - loss: 2.3226 - acc: 0.5071 - val_loss: 2.0728 - val_acc: 0.5270
Epoch 5/50
73892/73892 [==============================] - 2s 23us/step - loss: 1.8639 - acc: 0.5672 - val_loss: 1.7036 - val_acc: 0.5964
Epoch 6/50
73892/73892 [==============================] - 2s 24us/step - loss: 1.5963 - acc: 0.6252 - val_loss: 1.5062 - val_acc: 0.6414
Epoch 7/50
73892/73892 [==============================] - 2s 24us/step - loss: 1.4271 - acc: 0.6614 - val_loss: 1.3540 - val_a

#### Evaluate Test Accuracy

In [150]:
tmp_x = pad(test_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length))

_loss, _acc = simple_embed_model.evaluate(tmp_x, test_french_sentences, batch_size=1024, verbose=True)
print(f'loss: {_loss:.3f}, accuracy: {_acc:.3f}')

45495/45495 [==============================] - 0s 8us/step
loss: nan, accuracy: 0.905


### Model 3: Bidirectional RNNs (IMPLEMENTATION)
![RNN](images/bidirectional.png)
One restriction of a RNN is that it can't see the future input, only the past.  This is where bidirectional recurrent neural networks come in.  They are able to see the future data.

In [151]:
def bd_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a bidirectional RNN model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3

    input_seq = Input(input_shape[1:])
    rnn = Bidirectional(GRU(64, return_sequences=True))(input_seq)
    logits = TimeDistributed(Dense(french_vocab_size))(rnn)

    model = Model(input_seq, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_bd_model(bd_model)

# TODO: Reshape the input
tmp_x = pad(train_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# TODO: Train the neural network
simple_bd_model = bd_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_bd_model.fit(tmp_x, train_french_sentences, batch_size=1024, epochs=50, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(simple_bd_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 73892 samples, validate on 18474 samples
Epoch 1/50
73892/73892 [==============================] - 15s 200us/step - loss: 3.5562 - acc: 0.4451 - val_loss: 2.4211 - val_acc: 0.4826
Epoch 2/50
73892/73892 [==============================] - 3s 35us/step - loss: 2.2075 - acc: 0.5183 - val_loss: 1.9891 - val_acc: 0.5625
Epoch 3/50
73892/73892 [==============================] - 3s 36us/step - loss: 1.8134 - acc: 0.5727 - val_loss: 1.6862 - val_acc: 0.5821
Epoch 4/50
73892/73892 [==============================] - 3s 36us/step - loss: 1.6081 - acc: 0.5964 - val_loss: 1.5513 - val_acc: 0.6049
Epoch 5/50
73892/73892 [==============================] - 3s 35us/step - loss: 1.5014 - acc: 0.6118 - val_loss: 1.4669 - val_acc: 0.6161
Epoch 6/50
73892/73892 [==============================] - 3s 35us/step - loss: 1.4298 - acc: 0.6213 - val_loss: 1.4066 - val_acc: 0.6233
Epoch 7/50
73892/73892 [==============================] - 3s 36us/step - loss: 1.3751 - acc: 0.6294 - val_loss: 1.3570 - val_a

#### Evaluate Test Accuracy

In [152]:
tmp_x = pad(test_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

_loss, _acc = simple_bd_model.evaluate(tmp_x, test_french_sentences, batch_size=1024, verbose=True)
print(f'loss: {_loss:.3f}, accuracy: {_acc:.3f}')

45495/45495 [==============================] - 1s 11us/step
loss: nan, accuracy: 0.715


### Model 4: Encoder-Decoder (IMPLEMENTATION)
Time to look at encoder-decoder models.  This model is made up of an encoder and decoder. The encoder creates a matrix representation of the sentence.  The decoder takes this matrix as input and predicts the translation as output.

Create an encoder-decoder model in the cell below.

In [153]:
def encdec_model(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train an encoder-decoder model on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # TODO: Implement
    learning_rate = 1e-3
    latent_dim = 64

    encoder_inputs = Input(input_shape[1:])
    encoder_output, state_h = GRU(latent_dim, return_state=True)(encoder_inputs)
    
    decoder_inputs = RepeatVector(output_sequence_length)(encoder_output)
    decoder_output = GRU(latent_dim, return_sequences=True)(decoder_inputs, initial_state=state_h)
    logits = TimeDistributed(Dense(french_vocab_size))(decoder_output)

    model = Model(encoder_inputs, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model
    
tests.test_encdec_model(encdec_model)

# TODO: Reshape the input
tmp_x = pad(train_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

# TODO: Train the neural network
simple_encdec_model = encdec_model(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_encdec_model.fit(tmp_x, train_french_sentences, batch_size=1024, epochs=50, validation_split=0.2)

# TODO: Print prediction(s)
print(logits_to_text(simple_encdec_model.predict(tmp_x[:1])[0], french_tokenizer))

Train on 73892 samples, validate on 18474 samples
Epoch 1/50
73892/73892 [==============================] - 15s 208us/step - loss: 3.7224 - acc: 0.4158 - val_loss: 2.7637 - val_acc: 0.4416
Epoch 2/50
73892/73892 [==============================] - 3s 39us/step - loss: 2.6043 - acc: 0.4629 - val_loss: 2.4968 - val_acc: 0.4709
Epoch 3/50
73892/73892 [==============================] - 3s 39us/step - loss: 2.3947 - acc: 0.4922 - val_loss: 2.3356 - val_acc: 0.4960
Epoch 4/50
73892/73892 [==============================] - 3s 39us/step - loss: 2.2871 - acc: 0.4997 - val_loss: 2.2590 - val_acc: 0.5009
Epoch 5/50
73892/73892 [==============================] - 3s 39us/step - loss: 2.2181 - acc: 0.5022 - val_loss: 2.1872 - val_acc: 0.5064
Epoch 6/50
73892/73892 [==============================] - 3s 39us/step - loss: 2.1304 - acc: 0.5122 - val_loss: 2.0840 - val_acc: 0.5154
Epoch 7/50
73892/73892 [==============================] - 3s 39us/step - loss: 2.0016 - acc: 0.5207 - val_loss: 1.9334 - val_a

#### Evaluate Test Accuracy

In [155]:
tmp_x = pad(test_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length, 1))

_loss, _acc = simple_encdec_model.evaluate(tmp_x, test_french_sentences, batch_size=1024, verbose=True)
print(f'loss: {_loss:.3f}, accuracy: {_acc:.3f}')

45495/45495 [==============================] - 1s 12us/step
loss: nan, accuracy: 0.667


### Model 5: Custom (IMPLEMENTATION)
Use everything you learned from the previous models to create a model that incorporates embedding and a bidirectional rnn into one model.

In [156]:
def model_final(input_shape, output_sequence_length, english_vocab_size, french_vocab_size):
    """
    Build and train a model that incorporates embedding, encoder-decoder, and bidirectional RNN on x and y
    :param input_shape: Tuple of input shape
    :param output_sequence_length: Length of output sequence
    :param english_vocab_size: Number of unique English words in the dataset
    :param french_vocab_size: Number of unique French words in the dataset
    :return: Keras model built, but not trained
    """
    # # TODO: Implement
    learning_rate = 1e-3
    embedding_size = 32
    latent_dim = 64

    encoder_inputs = Input(input_shape[1:])
    encoder_emb = Embedding(english_vocab_size, embedding_size, input_length=input_shape[1])(encoder_inputs)
    encoder_output, state_h1, state_h2 = Bidirectional(GRU(latent_dim, return_state=True))(encoder_emb)

    # decoder_inputs = RepeatVector(output_sequence_length)(encoder_output)
    decoder_inputs = RepeatVector(output_sequence_length)(encoder_output)
    decoder_output = Bidirectional(GRU(latent_dim, return_sequences=True))(decoder_inputs, initial_state=[state_h1, state_h2])
    logits = TimeDistributed(Dense(french_vocab_size))(decoder_output)

    model = Model(encoder_inputs, Activation('softmax')(logits))
    model.compile(loss=sparse_categorical_crossentropy,
                  optimizer=Adam(learning_rate),
                  metrics=['accuracy'])
    return model

tests.test_model_final(model_final)

# TODO: Reshape the input
tmp_x = pad(train_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length))

print('Final Model Loaded')
# TODO: Train the final model
simple_final_model = model_final(
    tmp_x.shape,
    max_french_sequence_length,
    english_vocab_size,
    french_vocab_size)
simple_final_model.fit(tmp_x, train_french_sentences, batch_size=1024, epochs=50, validation_split=0.2)


Final Model Loaded
Train on 73892 samples, validate on 18474 samples
Epoch 1/50
73892/73892 [==============================] - 19s 255us/step - loss: 3.7020 - acc: 0.4044 - val_loss: 2.7925 - val_acc: 0.4404
Epoch 2/50
73892/73892 [==============================] - 5s 71us/step - loss: 2.4647 - acc: 0.4715 - val_loss: 2.2598 - val_acc: 0.5003
Epoch 3/50
73892/73892 [==============================] - 5s 71us/step - loss: 2.1323 - acc: 0.5158 - val_loss: 1.9586 - val_acc: 0.5282
Epoch 4/50
73892/73892 [==============================] - 5s 71us/step - loss: 1.8377 - acc: 0.5470 - val_loss: 1.7506 - val_acc: 0.5538
Epoch 5/50
73892/73892 [==============================] - 5s 71us/step - loss: 1.6801 - acc: 0.5687 - val_loss: 1.6302 - val_acc: 0.5748
Epoch 6/50
73892/73892 [==============================] - 5s 71us/step - loss: 1.5713 - acc: 0.5858 - val_loss: 1.5296 - val_acc: 0.5893
Epoch 7/50
73892/73892 [==============================] - 5s 71us/step - loss: 1.4838 - acc: 0.6015 - val_l

#### Evaluate Test Accuracy

In [157]:
tmp_x = pad(test_english_sentences, max_french_sequence_length)
tmp_x = tmp_x.reshape((-1, max_french_sequence_length))

_loss, _acc = simple_final_model.evaluate(tmp_x, test_french_sentences, batch_size=1024, verbose=True)
print(f'loss: {_loss:.3f}, accuracy: {_acc:.3f}')

45495/45495 [==============================] - 1s 17us/step
loss: nan, accuracy: 0.789


## Prediction (IMPLEMENTATION)

In [137]:
def final_predictions(x, y, x_tk, y_tk):
    """
    Gets predictions using the final model
    :param x: Preprocessed English data
    :param y: Preprocessed French data
    :param x_tk: English tokenizer
    :param y_tk: French tokenizer
    """
    # TODO: Train neural network using model_final
    model = simple_final_model
    
    ## DON'T EDIT ANYTHING BELOW THIS LINE
    y_id_to_word = {value: key for key, value in y_tk.word_index.items()}
    y_id_to_word[0] = '<PAD>'

    sentence = 'he saw a old yellow truck'
    sentence = [x_tk.word_index[word] for word in sentence.split()]
    sentence = pad_sequences([sentence], maxlen=x.shape[-1], padding='post')
    sentences = np.array([sentence[0], x[0]])
    sentences = pad_sequences(sentences, max_french_sequence_length)
    predictions = model.predict(sentences, len(sentences))

    print('Sample 1:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[0]]))
    print('Il a vu un vieux camion jaune')
    print('Sample 2:')
    print(' '.join([y_id_to_word[np.argmax(x)] for x in predictions[1]]))
    print(' '.join([y_id_to_word[np.max(x)] for x in y[0]]))


final_predictions(preproc_english_sentences, preproc_french_sentences, english_tokenizer, french_tokenizer)

Sample 1:
la fruit est fruit la mais mais est favori <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
Il a vu un vieux camion jaune
Sample 2:
les est est en en en en en en <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>
new jersey est parfois calme pendant l' automne et il est neigeux en avril <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD>


## Submission
When you're ready to submit, complete the following steps:
1. Review the rubric to ensure your submission meets all requirements to pass
2. Generate an HTML version of this notebook

  - Run the next cell to attempt automatic generation (this is the recommended method in Workspaces)
  - Navigate to **FILE -> Download as -> HTML (.html)**
  - Manually generate a copy using `nbconvert` from your shell terminal
```
$ pip install nbconvert
$ python -m nbconvert machine_translation.ipynb
```
  
3. Submit the project

  - If you are in a Workspace, simply click the "Submit Project" button (bottom towards the right)
  
  - Otherwise, add the following files into a zip archive and submit them 
  - `helper.py`
  - `machine_translation.ipynb`
  - `machine_translation.html`
    - You can export the notebook by navigating to **File -> Download as -> HTML (.html)**.

### Generate the html

**Save your notebook before running the next cell to generate the HTML output.** Then submit your project.

In [2]:
# Save before you run this cell!
!!jupyter nbconvert *.ipynb

['[NbConvertApp] Converting notebook machine_translation.ipynb to html',
 '[NbConvertApp] Writing 305996 bytes to machine_translation.html']

## Optional Enhancements

This project focuses on learning various network architectures for machine translation, but we don't evaluate the models according to best practices by splitting the data into separate test & training sets -- so the model accuracy is overstated. Use the [`sklearn.model_selection.train_test_split()`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html) function to create separate training & test datasets, then retrain each of the models using only the training set and evaluate the prediction accuracy using the hold out test set. Does the "best" model change?